In [1]:
!pip install psycopg2

In [4]:
import sys
sys.path.append('..\\sif_research\\data_utils')

import sif_utils as su
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import importlib
import seaborn # optional
seaborn.set() # optional
plt.rcParams['figure.figsize'] = [10, 6] # increase figure size


# Adds the research_framework folder to system path so python knows to look their for imports
sys.path.append('..\\sif_research\\research_framework') # CHANGE FOR MAC OS

import metrics

## Use code from backtest example to load data ##

In [9]:
# Global Variables / Research Environment Settings
start_date = datetime.datetime(day=3, month=1, year=2012)
end_date = datetime.datetime(day=1, month=1, year=2017)
days = (end_date - start_date).days

# Number of stocks to have in the universe
top_N = 100

In [10]:
# Grab all tickers in the universe
tickers = sorted(su.get_universe_tickers(top_N, start_date, end_date))

In [11]:
# Pull data needed for research and store into variables, or however
# you want to handle your data.

# This is the order of the parameters, once you are comfortable feel free
# to not use paramater=foo and just do foo, bar, ...

price_attrs = ['open', 'close', 'low', 'high']
data = su.get_equity_data(tickers=tickers, start_date=start_date, end_date=end_date,
                          attributes=price_attrs)

In [12]:
# Convert from dictionary of dataframes to numpy arrays

OPEN = data['open'].astype(float).values
CLOSE = data['close'].astype(float).values
LOW = data['low'].astype(float).values
HIGH = data['high'].astype(float).values

In [13]:
universe_filter = su.create_universe_filter(top_N, start_date, end_date, tickers, data['close'].index.values)

In [14]:
# Fills all the tickers not in the universe at that point with 0
FILLED_CLOSE = metrics.fill_nonuniverse(universe_filter.values, CLOSE, fill_val=0)

## My strategy going here ##

In [15]:
def strategy(day):
    """
    Attempting to implement basic mean reversion strategy
    :return: numpy array of shape: 1 x len(Universe) with equity holdings
    """
   

In [16]:
# compute holdings
universe_size = len(tickers)
holdings = metrics.backtest(strategy, CLOSE.shape[0], universe_size, lookback)
print(f'Holdings Shape: {holdings.shape}')

NameError: name 'lookback' is not defined

## Starting again with stuff from Hao's notebook ##

In [4]:
!pip install psycopg2

In [1]:
import sys
import pandas as pd
import numpy as np
import datetime
sys.path.append('..\\sif_research\\research_framework')
import metrics
import os
from operators import rank, ts_rank, group_rank, weight


sys.path.append('..\\sif_research\\data_utils')

import sif_utils as su

In [15]:
# Global Variables / Research Environment Settings
start_date = datetime.datetime(day=1, month=1, year=2010)
end_date = datetime.datetime(day=1, month=1, year=2018)
days = (end_date - start_date).days
lookback = 20
delay = 1
top_N = 300
initial_investment = 1000
save_backtest_result = True
backtest_result_folder = './backtest_result'
# Get all tickers
tickers = sorted(su.get_universe_tickers(top_N, start_date, end_date))

In [10]:
# Grab all tickers in the universe and sort them
tickers = sorted(su.get_universe_tickers(top_N, start_date, end_date))
universe_size = len(tickers)

price_attrs = ['open', 'close', 'low', 'high', 'volume']
data = su.get_equity_data(tickers=tickers, start_date=start_date,
                          end_date=end_date, attributes=price_attrs)


In [11]:
OPEN = data['open'].astype(float).values
CLOSE = data['close'].astype(float).values
LOW = data['low'].astype(float).values
HIGH = data['high'].astype(float).values
VOLUME = data['volume'].astype(float).values

# Create universe filter table
all_dates = data['close'].index.values
universe_filter = su.create_universe_filter(top_N, start_date, end_date, tickers, all_dates)

# Grab all tickers in the universe and sort them
tickers = sorted(su.get_universe_tickers(top_N, start_date, end_date))
universe_size = len(tickers)
universe_filter_arr = universe_filter.values

In [5]:
data.keys()

dict_keys(['open', 'close', 'low', 'high', 'volume'])

In [6]:
# Hao's example alpha
def alpha(day):
    # raw factor computed for all tickers (not necesarily the valid universe)
    factor = -rank(CLOSE[day - delay] - CLOSE[day - delay - 4])
    return weight(factor, universe_filter_arr[day])

In [7]:
# See Zura et al. 151 Trading Strategies Section 3.1
def momentum_alpha(day):
    momentum = rank((CLOSE[day - delay] / CLOSE[day - delay - 30]))
    return weight(momentum, universe_filter_arr[day])

In [17]:
def skew_momentum_alpha(day):
    # Using the max daily return from the past 3 months to represent skew
#     print(day)
    skew_start = day - delay - 29
    skew_end = day
#     print(f'Skew start: {skew_start} , skew end: {skew_end}')
    daily_rets = CLOSE[skew_start:skew_end] - OPEN[skew_start:skew_end]
    
    skew = rank(np.amax(daily_rets, axis=0))

    # Using return from the past 3 months as momentum
    momentum = rank(CLOSE[day - delay] / CLOSE[day - delay - 252])

    # We believe that stocks with either high skew and negative momentum OR stocks with low skew and positive momentum are the stocks we want to long
    # These stocks are said to have "enhanced momentum"
    # High skew neg momentum signifies a stock that is currently undervalued?, as in its current pricing represents a negative outlier value and it will revert to the mean
    # Low skew pos momentum signifies a stock that is currently performing well and appears to be performing around its average, therefore this stock is a good buy as 
    #    we expect it to continue on this trend

    # Simple and likely nonoptimal way to try to find these stocks with mixed skew/momentum high/low or low/high is to add their ranked arrays together
        # Side note, could weight the momentum vs. skew arrays differently to see if that changes anything
    # Since the rankings are both equal, (rank assigns a percentile to each ticker), The combination of skew/momentum high/high will be closer to 2 and low/low will be closer to 0, 
    # so we want middling values

    # In a simple effort to address this, we rank the values in the combined array of skew and momentum by each elements absolute difference from 1
    # (i.e. -rank by abs diff to ensure that the lowest abs diff is ranked highest)
#         combined = skew + momentum
#     #     print(combined)
#         factor = -rank(abs(combined - 1))
#         return weight(factor, universe_filter_arr[day])

    # Trying paper's approach for ranking
    # Take stocks that are in the top quintile of skew and bottom quintile of momentum or in the bottom quintile of skew and top quintile of momentum
    factor = np.zeros(len(skew))
#     print(skew)
#     print(momentum)
    for i in range(0, len(skew)):
        if((skew[i] >= .8 and momentum[i] <= .2) or (skew[i] <= .2 and momentum[i] >= .8)):
            # Long the stock
            factor[i] = 1
        else:
            factor[i] = 0
    # Inverting the factor gives much better returns???
    # Did I get something backwards???
    return weight(-factor, universe_filter_arr[day])

In [191]:
def long_all(day):
    all_long = np.zeros(318)
    all_long.fill(1/300)
    return all_long

In [192]:
long_all(346)

array([0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333333,
       0.00333333, 0.00333333, 0.00333333, 0.00333333, 0.00333

In [80]:
# Can do something with close to open overnight
def overnight(day):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-80-d8fb8771adf2>, line 3)

### Some notes from meeting 2/24/2020 to possibly make alphas ### 

 Momentum again from video from 2/24/2020
 
 Look at convexity and concavity
 
 y = Gain * t + Acc * x^2
 
 Accelerated vs decelerated gain
 
 Whether coefficients are positive or negative
 
 Approximating curves with polynomials

 Skewness and momentum
 Skewness as a signal for stocks that are overbrought

Proxy for skew, max daily return from past 20 days/ past month (positive skew)
    Could also do negative skew with minimum

Assume return over trailing year as momentum

More strongly positive skew and positive momentum represents weakened momentum

Positive momentum and less skew is enhanced momentum

Negativee momentum and positive skew is enhanced momentum

Negative momenutum and less skew is weakened momentum

Ideas for fundamanetal frm Hao
   
   Rank ( equity/ assets) <-- cross sectional
   
   ts_rank(-debt/equity, 252 <-- or whatever lookback) <-- ts_info
   
   abs(eps[day - 1] - eps[day - 2]) > 0 <-- check to see if comany updated financial statements
   
       Include mean reversion/momentum factor and combine with this to build an alpha

In [18]:
# run backtest, print the performance table and draw the chart
# For skew momentum
lookback = 252
holdings = metrics.backtest(skew_momentum_alpha, CLOSE.shape[0], universe_size, lookback)
daily_returns = metrics.daily_return(holdings[:-1], OPEN[lookback:] )
performance = metrics.generate_performance_table(daily_returns, holdings,
                                                 dates=all_dates[lookback:-1],
                                                tickers=tickers)
print(performance)
pnl_df = pd.DataFrame(daily_returns.cumsum() * initial_investment,index = data['close'].iloc[lookback:-1].index, columns = ['alpha'])
chart = pnl_df.plot(grid=True,figsize=[12,8],linewidth=1,color='darkorange')


ValueError: Length mismatch: Expected axis has 526 elements, new values have 517 elements

In [ ]:
# save backtest result
if save_backtest_result == True:
    if not os.path.exists(backtest_result_folder):
        os.makedirs(backtest_result_folder)
    pnl_df.to_csv(backtest_result_folder+'/'+'pnl.csv')
    pd.DataFrame(holdings[:-1],index = data['close'].iloc[lookback:-1].index, columns = tickers).to_csv(backtest_result_folder+'/'+'holdings.csv')
    chart.get_figure().savefig(backtest_result_folder+'/'+'performance.png')
        